In [1]:
#!pip install
# !pip install pickle5
!python --version

Python 3.9.16


# import

In [2]:
import cv2
import mediapipe as mp
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector
import numpy as np
import time
from datetime import datetime
import utils,math
import matplotlib.pyplot as plt
import pandas as pd
import os
import mss
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import sys
# import tqdm
import json
import pickle
mp_face_mesh = mp.solutions.face_mesh
# face_mesh = mp_face_mesh.FaceMesh(
#     min_detection_confidence=0.5,
#     min_tracking_confidence=0.5,
#     # เพิ่มจำนวน detect หน้าสูงสุด
#     max_num_faces=10,
# )

mp_drawing = mp.solutions.drawing_utils

drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
count = 0
seconds = time.time()
# cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture('video/source/2/CB_SOS_11042022.mp4')

ith_sample = 0
face_frame = 0
status = list('-'*20)

# variables 
CEF_COUNTER =0
TOTAL_BLINKS =0
# constants
CLOSED_EYES_FRAME =3
mouthStatus = ''
# *-------------------------------*
#screen size
# monitor = {"top": 40, "left": 40, "width": 500, "height": 500}
try:
    if len(sys.argv) == 1:
        monitor = {"top": 100, "left": 0, "width": 1500, "height": 2000}
    else:
        mornitor_t = int(sys.argv[-1].split(',')[1])
        mornitor_l = int(sys.argv[-1].split(',')[0])
        mornitor_w = int(sys.argv[-1].split(',')[2])
        mornitor_h = int(sys.argv[-1].split(',')[3])
        monitor = {"top": mornitor_t, "left": mornitor_l, "width": mornitor_w, "height": mornitor_h}
except:
        # monitor = {"top": 100, "left": 0, "width": 1280, "height": 720}
        monitor = {"top": 300, "left": 100, "width": 600, "height": 720}
        
# Left eyes indices 
LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
LEFT_EYEBROW =[ 336, 296, 334, 293, 300, 276, 283, 282, 295, 285 ]

# right eyes indices
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ]  
RIGHT_EYEBROW=[ 70, 63, 105, 66, 107, 55, 65, 52, 53, 46 ]
#data list csv

#eye
RIGHT_IRIS = [474, 475, 476, 477]
LEFT_IRIS = [469, 470, 471, 472]
L_H_LEFT = [33]  # right eye right most landmark
L_H_RIGHT = [133]  # right eye left most landmark
R_H_LEFT = [362]  # left eye right most landmark
R_H_RIGHT = [263]  # left eye left most landmark
avg = 0
#mount
detector = FaceMeshDetector(maxFaces=2)
idList = [0,17,78,292]  
ratio_mount = 0
mount_mean = []
#face
countF = 0
countL1 = 0
countL2 = 0
countR1 = 0
countR2 = 0
countU = 0
countD = 0
#eye
lookL = 0
lookR = 0
lookC = 0
blink = 0
len_mean = 25
check_blink = False
#mount
mount = 0
mean_mount = [] 

status_user = ""
fps = 0
predict = 0
X_test = []
X_train = []
flag_ready = False
# xy = [[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0]]
PATH_TO_MIDDLE = "output2.avi"
frame_width = 1280
frame_height = 720
frame_rate = 20.0
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(PATH_TO_MIDDLE, fourcc, frame_rate, 
                      (frame_width, frame_height))
get_data = []
#mos
count_cheat = 0
json_string = json.dumps(count_cheat)

#cal cheat
list_data_cheat = [0] * 3000
pre_x_cheat = 0
x_cheat = 0
hold_x_cheat = 0

#criterion
criterion_y = None

# load model

In [3]:
pkl_name = 'model/pkl_model2.pkl'
with open(pkl_name,'rb') as file:
    pickle_model = pickle.load(file)
    print(pickle_model)

# score = pickle_model.score(X,y)
# print('score {0:.2f}%'.format(100 * score))
# yPredict = pickle_model.predict(np.array(xy))
# print(yPredict)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(10, 5, 5), max_iter=1000,
              random_state=1, verbose=True)


# function

In [4]:
import csv
header = ['h_forward', 'h_left_1', 'h_left_2', 'h_right_1', 'h_right_2', 'h_up', 'h_down','L_left', 'L_Right', 'L_center', 'blink', 'mount', 'predict']
header2 = [['Time','Num face',"status user",'head_pose_forward','head_pose_left', 'head_pose_right', 'head_pose_up', 'head_pose_down','look_left', 'look_Right', 'look_center', 'blink', 'mount']]
#weigh,anwer,frame,fps,
data_train = [
    [countF, countL1,countL2,  countR1, countR2, countU,  countD, lookL, lookR, lookC, blink, mount, predict],
]
def average(lst):
    return sum(lst) / len(lst)

def build_CSV_for_predict():
    with open('data_train_test2.csv', 'w') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        # Use writerows() not writerow()
        writer.writerows(data_train)

def build_CSV_for_watch(data_watch_csv):
    with open('csv/send/data_train_test4.csv', 'a', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        # writer.writerow(header)
        # Use writerows() not writerow()
        writer.writerows(data_watch_csv)        
        
        
def landmarksDetection(img, landmark, draw=False):
        img_height, img_width= img.shape[:2]
        # list[(x,y), (x,y)....]
        mesh_coord = [(int(point.x * img_width), int(point.y * img_height)) for point in landmark]
        if draw :
            [cv2.circle(img, p, 2, (0,255,0), -1) for p in mesh_coord]

        # returning the list of tuples for each landmarks 
        return mesh_coord

def saveJson():
    # open a file for writing
    with open('data.json', 'w') as file:
        file.truncate(0)
        # convert the count_cheat dictionary to a JSON string
        json_string = json.dumps(count_cheat)
        # write the JSON string to the file
        file.write(json_string)
    
    
# Euclaidean distance 
def euclaideanDistance(point, point1):
    x, y = point
    x1, y1 = point1
    distance = math.sqrt((x1 - x)**2 + (y1 - y)**2)
    return distance

# Blinking Ratio
def blinkRatio(img, landmarks, right_indices, left_indices):
    # Right eyes 
    # horizontal line 
    rh_right = landmarks[right_indices[0]]
    rh_left = landmarks[right_indices[8]]
    # vertical line 
    rv_top = landmarks[right_indices[12]]
    rv_bottom = landmarks[right_indices[4]]
    # draw lines on right eyes 
    # cv.line(img, rh_right, rh_left, utils.GREEN, 2)
    # cv.line(img, rv_top, rv_bottom, utils.WHITE, 2)

    # LEFT_EYE 
    # horizontal line 
    lh_right = landmarks[left_indices[0]]
    lh_left = landmarks[left_indices[8]]

    # vertical line 
    lv_top = landmarks[left_indices[12]]
    lv_bottom = landmarks[left_indices[4]]

    rhDistance = euclaideanDistance(rh_right, rh_left)
    rvDistance = euclaideanDistance(rv_top, rv_bottom)

    lvDistance = euclaideanDistance(lv_top, lv_bottom)
    lhDistance = euclaideanDistance(lh_right, lh_left)

    reRatio = rhDistance/rvDistance
    leRatio = lhDistance/lvDistance

    ratio = (reRatio+leRatio)/2
    return ratio 


# Eyes Extrctor function,
#Funcao que define distancias euclidianas dos pontos nos olhos
def euclidean_distance(point1, point2):
    x1, y1 =point1.ravel()
    x2, y2 =point2.ravel()
    distance = math.sqrt((x2-x1)**2 + (y2-y1)**2)
    return distance

#Funcao para encontra posicao da iris
def iris_position(iris_center, right_point, left_point):
    center_to_right_dist = euclidean_distance(iris_center, right_point)
    total_distance = euclidean_distance(right_point, left_point)
    ratio = center_to_right_dist/total_distance
    iris_position =""
    if check_blink == True:
        iris_position="center"
    elif ratio <= 0.42:
        iris_position="left"
    elif ratio > 0.42 and ratio <= 0.57:
        iris_position="center"
    else:
        iris_position = "right"
    return iris_position, ratio



build_CSV_for_predict()

In [5]:
criteria = {
    800: 10,
    900: 8,
    1000: 7,
    1100: 6,
    1300: 5
}
for width, criterion in criteria.items():
    if monitor['width'] < width:
        criterion_y = criterion
        break
else:
    criterion_y = 4  # Default criterion value if no match is found

print(criterion_y)

10


# use

In [6]:
# landmark detection function 
# if(monitor['width'] < 800):
#     criterion_y = 10
#     print(10)
# elif(monitor['width'] >= 800 and monitor['width'] < 900):
#     criterion_y = 8
#     print(8)
# elif(monitor['width'] >= 900 and monitor['width'] < 1000):
#     criterion_y = 7
#     print(7)
# elif(monitor['width'] >= 1000 and monitor['width'] < 1100):
#     criterion_y = 6
#     print(6)
# elif(monitor['width'] >= 1100 and monitor['width'] < 1300):
#     criterion_y = 5
#     print(5)
# else:
#     criterion_y = 4
#     print(4)
    
    
build_CSV_for_watch(header2)
print(header)
with mp_face_mesh.FaceMesh(max_num_faces=10, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    with mss.mss() as sct: #แคปหน้าจอ
        success = True
        print(mss.mss())
        while "Screen capturing":
           
            image = np.array(sct.grab(monitor))
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            ith_sample += 1
            start = time.time()
            image, faces = detector.findFaceMesh(image,draw = False)
            
            image.flags.writeable = False

            # Get the result
            results = face_mesh.process(image)

            # To improve performance
            image.flags.writeable = True

            # Convert the color space from RGB to BGR
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            frame = image
            frame = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            img_h, img_w, img_c = image.shape
            face_3d = []
            face_2d = []
            

            # ดูจำนวนคน
            # print('len(results.multi_face_landmarks)')
            cv2.putText(image, 'frame '+str(ith_sample), (20, 500), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)
            cv2.putText(frame, 'frame '+str(ith_sample), (20, 500), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)
            if results.multi_face_landmarks: #เจอหน้าคน
                face_frame += 1
                pre_x_cheat = x_cheat 
                cv2.putText(image,'face frame '+ str(face_frame), (20, 550), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)  
                # classIds, confs, bbox = net.detect(image,confThreshold=0.5)
                # for classId, confidence, box in zip(classIds.flatten(),confs.flatten(),bbox):
                #     cv2.putText(image,classNames[classId-1],(box[0]+10,box[1]+30),cv2.FONT_HERSHEY_COMPLEX,2,(0,255,0),2)
                    
                
                
                mesh_points=np.array([np.multiply([p.x, p.y], [img_w, img_h]).astype(int) for p in results.multi_face_landmarks[0].landmark])
                (l_cx, l_cy), l_radius = cv2.minEnclosingCircle(mesh_points[LEFT_IRIS])
                (r_cx,r_cy), r_radius = cv2.minEnclosingCircle(mesh_points[RIGHT_IRIS])
                # transforma pontos centrais em array np
                center_left = np.array([l_cx, l_cy], dtype=np.int32)
                center_right = np.array([r_cx, r_cy], dtype=np.int32)
                #desenhe o círculo com base nos valores de retorno da minEnclosingCircle, através do CIRCLE que desenha a imagem do círculo com base no centro (x, y) e no raio
                cv2.circle(image, center_left, int(l_radius), (255, 0, 255), 1, cv2.LINE_AA)
                cv2.circle(image, center_right, int(r_radius), (255, 0, 255), 1, cv2.LINE_AA)
                #mostrar pontos nos cantos dos olhos
                cv2.circle(image, mesh_points[R_H_RIGHT][0], 3, (255, 255, 255), -1, cv2.LINE_AA)
                cv2.circle(image, mesh_points[R_H_LEFT][0], 3, (0, 255, 255), -1, cv2.LINE_AA)
                
               # ส่วนของการมอง
                try:
                    iris_pos, ratio = iris_position(center_right, mesh_points[R_H_RIGHT], mesh_points[R_H_LEFT][0])
                except:
                    print("iris_pos error")
              
                # cv2.putText(image, f"ref : {iris_pos} {ratio:.2f}",(50,50),cv2.FONT_HERSHEY_COMPLEX,0.7,(255,0,255),1)
                #eye//
                for i,face_landmarks in enumerate(results.multi_face_landmarks):
                    # เพิ่มกรอบหน้าขาว
                    mp_drawing.draw_landmarks(
                        image=image,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_CONTOURS,
                        landmark_drawing_spec=drawing_spec,
                        connection_drawing_spec=drawing_spec)
                    
                    # ส่วนของการกระพริบตา
                    mesh_coords = landmarksDetection(image, face_landmarks.landmark, False)
                    try:
                        ratio = blinkRatio(image, mesh_coords, RIGHT_EYE, LEFT_EYE)
                    except:
                        print("blink error")
                 
                    # ส่วนของปาก
                    try:
                        if faces:
                            face = faces[0]
                            for id in idList:
                                cv2.circle(image,face[id],5,(255,0,255),5)
                            upDown,_ = detector.findDistance(face[idList[0]],face[idList[1]])
                            leftRight,_ = detector.findDistance(face[idList[2]],face[idList[3]])
                            ratio_mount = int((upDown/leftRight)*100)
                            
                            if(len(mean_mount) < len_mean): #หาค่าเฉี่ลยปาก
                                mean_mount.append(ratio_mount)
                                avg = average(mean_mount)
                                # print(avg)
                            if(len(mean_mount)>= len_mean):
                                # print('avg ',avg)
                                # print('ratio_mount ',ratio_mount)
                                if ratio_mount > avg + 15:
                                    mouthStatus = 'open'
                                    cv2.line(image,face[idList[0]],face[idList[1]],(0,0,255),3)
                                    cv2.line(image,face[idList[2]],face[idList[3]],(0,0,255),3)
                                else:
                                    cv2.line(image,face[idList[0]],face[idList[1]],(0,255,0),3)
                                    cv2.line(image,face[idList[2]],face[idList[3]],(0,255,0),3)
                                    mouthStatus = 'close'
                    except:
                        print("mount err")
                        
                    # cv2.putText(image, f'Ratio : {round(ratio,2)}', (20, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                    if ratio > 6: # ตรวจการกระพริบของตา
                        CEF_COUNTER +=1
                        check_blink = True
                        # cv.putText(image, 'Blink', (200, 50), FONTS, 1.3, utils.PINK, 2)
                        utils.colorBackgroundText(image,  f'Blink', cv2.FONT_HERSHEY_SIMPLEX, 0.7, (20, 100), 2, utils.YELLOW, pad_x=6, pad_y=6,)

                    else:
                        check_blink = False
                        if CEF_COUNTER>CLOSED_EYES_FRAME:
                            TOTAL_BLINKS +=1
                            CEF_COUNTER =0
                            
                    face_2d = []
                    face_3d = []
                    for idx, lm in enumerate(face_landmarks.landmark):

                        # เอาไว้หาตำแหน่งจุดของหน้า
                        # test_x, test_y = int(lm.x * img_w), int(lm.y * img_h)
                        # cv2.circle(image, (test_x, test_y), 2, (255, 0, 0), -1)
                        # cv2.putText(image, str(idx), (test_x, test_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

                        # เก็บตำแหน่งบนหน้าผาก
                        if idx == 10:
                            header_x,header_y = int(lm.x * img_w), (int(lm.y * img_h))
                            color_nose = (255, 0, 0)
                        # เก็บตำแหน่งจมูก
                        if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
                            # print('mosss',i)
                            if idx == 1:
                                nose_2d = (lm.x * img_w, lm.y * img_h)
                                nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)
                            x, y = int(lm.x * img_w), int(lm.y * img_h)

                            # Get the 2D Coordinates
                            face_2d.append([x, y])

                            # Get the 3D Coordinates
                            face_3d.append([x, y, lm.z])

                    # Convert it to the NumPy array
                    face_2d = np.array(face_2d, dtype=np.float64)

                    # Convert it to the NumPy array
                    face_3d = np.array(face_3d, dtype=np.float64)

                    # The camera matrix
                    focal_length = 1 * img_w

                    cam_matrix = np.array([[focal_length, 0, img_h / 2],
                                           [0, focal_length, img_w / 2],
                                           [0, 0, 1]])

                    # The distortion parameters
                    dist_matrix = np.zeros((4, 1), dtype=np.float64)

                    # Solve PnP
                    success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)

                    # Get rotational matrix
                    rmat, jac = cv2.Rodrigues(rot_vec)

                    # Get angles
                    angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)

                    # Get the y rotation degree
                    x = angles[0] * 360
                    y = angles[1] * 360
                    z = angles[2] * 360

                    start_point = (5, 220)
                    end_point = (440, 440)
                    color = (255, 0, 0)
                    thickness = 2
                    
                    #reset val
                    countF, countL1, countL2, countR1, countR2, countU, countD, lookL, lookR, lookC, blink, mount, predict = [0] * 13
                    # See where the user's head tilting
                    if(ratio_mount > avg + 10):
                        # print('mount+1')
                        mount = 1
                    if(ratio > 6):
                        blink = 1
                    if(iris_pos == 'right'):
                        lookR = 1
                    elif(iris_pos == 'left'):
                        lookL = 1
                    else:
                        lookC = 1
                        
                    #angles[1] = y
                    #angles[0] = x
                    if y < -criterion_y: #5
                    # if angles[1] < -0.020:
                        text = "Looking Right"
                        count = count+1
                        if y < -(criterion_y + 3): #8
                            countR2 = 1
                        else:
                            countR1 = 1
                        if(iris_pos == 'right'):
                            # cv2.rectangle(image,(0,0),(image.shape[1],image.shape[0]),(0,0,255),10)
                            status_user = "red"
                        else:
                            # cv2.rectangle(image,(0,0),(image.shape[1],image.shape[0]),(0, 165, 255),10)
                            status_user = "orange"
                        color_nose = (0, 0, 255)
                    # elif angles[1] > 0.020:
                    elif y > criterion_y:
                        text = "Looking Left"
                        count = count+1    
                        if y > (criterion_y + 3):
                            countL2 = 1
                        else:
                            countL1 = 1
                        if(iris_pos == 'left'):
                            # cv2.rectangle(image,(0,0),(image.shape[1],image.shape[0]),(0,0,255),10)
                            status_user = "red"
                        else:
                            # cv2.rectangle(image,(0,0),(image.shape[1],image.shape[0]),(0, 165, 255),10)
                            status_user = "orange"
                        color_nose = (0, 0, 255)
                    elif x < -5:
                        text = "Looking Down"
                        countD = 1
                        # cv2.rectangle(image,(0,0),(image.shape[1],image.shape[0]),(0, 165, 255),10)
                        status_user = "orange"
                        color_nose = (0, 0, 255)
                    elif x > 6:
                        text = "Looking Up"
                        countU = 1
                        # cv2.rectangle(image,(0,0),(image.shape[1],image.shape[0]),(0,255,0),10)
                        status_user = "green"
                        color_nose = (255, 0, 0)
                    else:
                        text = "Forward"
                        count = count-1
                        countF = 1
                        if(iris_pos == 'right'):
                            # cv2.rectangle(image,(0,0),(image.shape[1],image.shape[0]),(0, 165, 255),10)
                            status_user = "orange"
                        elif(iris_pos == 'left'):
                            # cv2.rectangle(image,(0,0),(image.shape[1],image.shape[0]),(0, 165, 255),10)
                            status_user = "orange"
                        else:
                            # cv2.rectangle(image,(0,0),(image.shape[1],image.shape[0]),(0,255,0),10)
                            status_user = "green"
                        color_nose = (255, 0, 0)
                        
                    # prediction by AI
                    yPredict = None
                    temp = [countF, countL1, countL2,  countR1,  countR2, countU,  countD, lookL, lookR, lookC, blink, mount]
                    get_data.append(temp)
                    if((flag_ready == False) and (len(get_data) == 24)): 
                        flag_ready = True
                    elif flag_ready == True:
                        try: 
                            get_data.pop(0)
                            new_get_data = np.array(get_data)
                            yPredict = pickle_model.predict([new_get_data.ravel()])
                            x_cheat = yPredict
                            if(yPredict == 1):
                                predict = 1
                                cv2.rectangle(image,(0,0),(image.shape[1],image.shape[0]),(0,0,255),10)
                                cv2.rectangle(frame,(0,0),(image.shape[1],image.shape[0]),(0,0,255),10)
                            elif(yPredict == 0):
                                predict = 0
                                cv2.rectangle(image,(0,0),(image.shape[1],image.shape[0]),(0,255,0),10)
                                cv2.rectangle(frame,(0,0),(image.shape[1],image.shape[0]),(0,255,0),10)
                            # print(yPredict)
                        except:
                            print('prediction error')
                    
                    #get data     predict
                    temp2 = [countF, countL1, countL2,  countR1,  countR2, countU,  countD, lookL, lookR, lookC, blink, mount, predict]
                    data_train.append(temp2)
                    
                    # print(hold_x_cheat)
                    if (x_cheat == 1):
                        hold_x_cheat = 5
                    elif hold_x_cheat > 0:
                        x_cheat = 1
                        hold_x_cheat -= 1
                     
                    if(pre_x_cheat == 0 and x_cheat == 1):
                     
                        list_data_cheat.append(1)
                        list_data_cheat.pop(0)
                        
                    if predict == 1:
                        count_cheat = count_cheat + 1
                        saveJson()
                    
                    data_watch_csv = [[str(datetime.now()), len(results.multi_face_landmarks), status_user, countF, countL1,  countL2,  countR1, countR2, countU,  countD, lookL, lookR, lookC, blink, mount]]
                    # yPredict = pickle_model.predict(np.array(xy))
                    # print(yPredict)
                    
                   
                      
                       
                    # print(get_data)
                    
                    build_CSV_for_watch(data_watch_csv)
                    
                    
                    
                    
                    if(len(results.multi_face_landmarks) >= 2):
                        # cv2.rectangle(image,(0,0),(image.shape[1],image.shape[0]),(0,0,255),10)
                        status_user = "red"
                    if(ratio > 6):
                        if(angles[0] < -0.025):
                            # cv2.rectangle(image,(0,0),(image.shape[1],image.shape[0]),(0, 0, 255),10)
                            status_user = "red"
                        else:
                            # cv2.rectangle(image,(0,0),(image.shape[1],image.shape[0]),(0, 165, 255),10)
                            status_user = "orange"
                    

                    # Display the nose direction
                    nose_3d_projection, jacobian = cv2.projectPoints(nose_3d, rot_vec, trans_vec, cam_matrix, dist_matrix)

                    p1 = (int(nose_2d[0]), int(nose_2d[1]))
                    p2 = (int(nose_2d[0] + y * 10), int(nose_2d[1] - x * 10))
                    cv2.line(image, p1, p2, color_nose, 3)
                    # Add the text on the image
                    cv2.putText(image, text, (header_x,header_y), cv2.FONT_HERSHEY_SIMPLEX, 1, color_nose, 2)
                    cv2.putText(image, "x: " + str(np.round(x, 2)), (header_x,header_y-100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    cv2.putText(image, "y: " + str(np.round(y, 2)), (header_x,header_y-80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    cv2.putText(image, "z: " + str(np.round(z, 2)), (header_x,header_y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    cv2.putText(image, f"eye : {iris_pos} {ratio:.2f}",(header_x,header_y-20),cv2.FONT_HERSHEY_COMPLEX,0.7,(255,0,255),1)
                    cv2.putText(image, f"mouth : {mouthStatus} {ratio_mount:.2f}",(header_x,header_y-40),cv2.FONT_HERSHEY_COMPLEX,0.7,(255,0,255),1)



                end = time.time()
                totalTime = end - start

                fps = 1 / totalTime
                # print(fps)
                # print(str(datetime.now()))

                #cv2.putText(image, f'FPS: {int(fps)}', (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)
            else: #ไม่เจอหน้าคน
                data_watch_csv = [[str(datetime.now()), 0, "unknow", 0, 0,  0, 0,  0, 0, 0, 0, 0, 0]]
                build_CSV_for_watch(data_watch_csv)
                x_cheat = 1

                # temp = [countF, countL1, countL2 , countR1, countR2, countU,  countD, lookL, lookR, lookC, blink, mount, predict]
                temp =   [0,      0,        0 ,       0,       0,      0,       0,      0,     0,     0,     0,      0,    1]
                # count_cheat.append(predict)  
                count_cheat = count_cheat + 1
                if (x_cheat == 1):
                    hold_x_cheat = 5
                elif hold_x_cheat > 0:
                    x_cheat = 1
                    hold_x_cheat -= 1
                if(pre_x_cheat == 0 and x_cheat == 1):
                    pre_x_cheat = 1
                    list_data_cheat.append(1)
                    list_data_cheat.pop(0)
                # list_data_cheat.append(1)
                # list_data_cheat.pop(1)
                saveJson()
                
                data_train.append(temp)

                # yPredict = pickle_model.predict(np.array(xy))
                # print(yPredict)

                # prediction by AI

                yPredict = None
                get_data.append(temp)
                if((flag_ready == False) and (len(get_data) == 24)): 
                    flag_ready = True
                elif flag_ready == True:
                    try: 
                        get_data.pop(0)
                        new_get_data = np.array(get_data)
                        cv2.rectangle(image,(0,0),(image.shape[1],image.shape[0]),(0,0,255),10)
                        # print(yPredict)
                    except:
                        print('p er')
                
            
            out.write(frame)
            cv2.imshow('Head Pose Estimation1', image)

            if cv2.waitKey(5) & 0xFF == 27:
                # X_test.append(temp)
                print('break')
                break
        out.release()
        cv2.destroyAllWindows()
       


['h_forward', 'h_left_1', 'h_left_2', 'h_right_1', 'h_right_2', 'h_up', 'h_down', 'L_left', 'L_Right', 'L_center', 'blink', 'mount', 'predict']
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
blink error
p er
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
p er
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
prediction error
p

In [7]:
build_CSV_for_predict()


# print('build CSV')
# X_test = []
# X_test.append(temp)
# print(temp)
# print(X_test)
# monitor = {"top": 40, "left": 40, "width": 500, "height": 500}
# print(len(xy[0]))
# print(np.array(xy))
# yPredict = pickle_model.predict(np.array(xy))
# print(yPredict)

In [8]:
# print(len(get_data))
# print(get_data)
# new_get_data = np.array(get_data)
# print(new_get_data)
# print([new_get_data.ravel()])
# print(len(new_get_data.ravel()))
# yPredict = pickle_model.predict([new_get_data.ravel()])
# print(yPredict)
print(count_cheat)
print(hold_x_cheat)

6875
5


In [9]:
print(len(data_train))
print(len(list_data_cheat))
print(list_data_cheat)
print(sum(list_data_cheat))

14174
3000
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0